In [1]:
import sys
sys.path.append("/Users/ktakeda/workspace/kaggle/rsna-breast-cancer-detection")
import cv2
import matplotlib.pyplot as plt

from torch.utils.data import Dataset, DataLoader, Subset

from rsna.utility import load_data
from rsna.preprocess import df_preprocess, Transform
from rsna.dataset import RSNADatasetPNG
from rsna import preprocess as prep
import matplotlib.patches as patches


import numpy as np
from omegaconf import DictConfig, OmegaConf

from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

/Users/ktakeda/workspace/kaggle/rsna-breast-cancer-detection/rsna3.10/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
df_data = load_data("train", custom_path="/Users/ktakeda/workspace/kaggle/rsna-breast-cancer-detection/data/dicom2png_512")
df_data = df_preprocess(df_data, is_train=True, sampling="down")
df_data.head()

,patient_id,image_id,age,implant,path,cancer,laterality,view
0,15696,1216427802,65.0,0,/Users/ktakeda/workspace/kaggle/rsna-breast-ca...,0,1,3
1,9228,334114182,51.0,0,/Users/ktakeda/workspace/kaggle/rsna-breast-ca...,0,1,3
2,50234,1509636612,52.0,0,/Users/ktakeda/workspace/kaggle/rsna-breast-ca...,0,1,1
3,26178,1617112699,40.0,0,/Users/ktakeda/workspace/kaggle/rsna-breast-ca...,0,1,1
4,15350,1399405629,65.0,0,/Users/ktakeda/workspace/kaggle/rsna-breast-ca...,0,1,1


In [4]:
df_data[["patient_id"]]

,patient_id
0,15696
1,9228
2,50234
3,26178
4,15350
...,...
2311,9707
2312,9840
2313,9840
2314,9851


In [29]:
df_data = load_data("train", custom_path="/Users/ktakeda/workspace/kaggle/rsna-breast-cancer-detection/data/dicom2png_512")
df_data = df_preprocess(df_data, is_train=True, sampling="down")

kf = KFold(n_splits=4, shuffle=True, random_state=1993)
unique_patient_id = df_data["patient_id"].unique()
patient_id = df_data["patient_id"]
for train_idx, valid_idx in kf.split(unique_patient_id):
    train_patient_id, valid_patient_id = unique_patient_id[train_idx], unique_patient_id[valid_idx]
    
    is_train = patient_id.isin(train_patient_id)
    is_valid = patient_id.isin(valid_patient_id)
    print(len(is_train), len(is_valid))

    print(display(df_data[is_train]))
    print(display(df_data[is_valid]))
    break

2316 2316


,patient_id,image_id,age,implant,path,cancer,laterality,view
0,15696,1216427802,65.0,0,/Users/ktakeda/workspace/kaggle/rsna-breast-ca...,0,1,3
1,9228,334114182,51.0,0,/Users/ktakeda/workspace/kaggle/rsna-breast-ca...,0,1,3
2,50234,1509636612,52.0,0,/Users/ktakeda/workspace/kaggle/rsna-breast-ca...,0,1,1
3,26178,1617112699,40.0,0,/Users/ktakeda/workspace/kaggle/rsna-breast-ca...,0,1,1
4,15350,1399405629,65.0,0,/Users/ktakeda/workspace/kaggle/rsna-breast-ca...,0,1,1
...,...,...,...,...,...,...,...,...
2309,9559,1174804734,76.0,0,/Users/ktakeda/workspace/kaggle/rsna-breast-ca...,1,0,3
2310,9707,1590470368,56.0,0,/Users/ktakeda/workspace/kaggle/rsna-breast-ca...,1,1,1
2311,9707,1900919592,56.0,0,/Users/ktakeda/workspace/kaggle/rsna-breast-ca...,1,1,3
2312,9840,111786935,59.0,0,/Users/ktakeda/workspace/kaggle/rsna-breast-ca...,1,1,3


None


,patient_id,image_id,age,implant,path,cancer,laterality,view
7,27203,1338072637,41.0,1,/Users/ktakeda/workspace/kaggle/rsna-breast-ca...,0,1,3
8,50583,1800513001,65.0,0,/Users/ktakeda/workspace/kaggle/rsna-breast-ca...,0,1,1
11,37512,1074178927,57.0,0,/Users/ktakeda/workspace/kaggle/rsna-breast-ca...,0,0,3
16,64883,1895558234,53.0,0,/Users/ktakeda/workspace/kaggle/rsna-breast-ca...,0,0,3
22,44868,720917760,53.0,0,/Users/ktakeda/workspace/kaggle/rsna-breast-ca...,0,1,1
...,...,...,...,...,...,...,...,...
2296,9167,402545081,62.0,0,/Users/ktakeda/workspace/kaggle/rsna-breast-ca...,1,0,1
2297,9167,436533616,62.0,0,/Users/ktakeda/workspace/kaggle/rsna-breast-ca...,1,0,1
2298,9167,906855351,62.0,0,/Users/ktakeda/workspace/kaggle/rsna-breast-ca...,1,0,3
2314,9851,1409270544,58.0,0,/Users/ktakeda/workspace/kaggle/rsna-breast-ca...,1,0,3


None


In [23]:
display(patient_id)
patient_id.isin(np.array(["9840"]))

0       15696
1        9228
2       50234
3       26178
4       15350
        ...  
2311     9707
2312     9840
2313     9840
2314     9851
2315     9851
Name: patient_id, Length: 2316, dtype: object

0       False
1       False
2       False
3       False
4       False
        ...  
2311    False
2312     True
2313     True
2314    False
2315    False
Name: patient_id, Length: 2316, dtype: bool

In [18]:
print(len(df_data))
print(df_data["patient_id"].unique(), len(df_data["patient_id"].unique()) )

2316
['15696' '9228' '50234' ... '9707' '9840' '9851'] 1554


In [8]:
type(group_fold), type(k_folds)

(sklearn.model_selection._split.GroupKFold, generator)

In [16]:
for train_index, valid_index in k_folds:
    pass
    print(train_index)
    break

[   0    5    6 ... 2311 2314 2315]


In [13]:
train_idnex

NameError: name 'train_idnex' is not defined